In [1]:
from sec import stock, lookups, processor
from typing import Iterator
from datetime import date, timedelta
import pandas as pd
from tqdm import tqdm

### 0. Update processed financial data

In [2]:
processor.download_sec_data(force_update=True)

Unzipping companyfacts.zip...


In [3]:
def daterange(start_date: str, end_date: str, interval: int = 1) -> Iterator[date]:
    """
    Generate a range of dates between start_date and end_date.
    
    Args:
        start_date (str): Start date in YYYY-MM-DD format.
        end_date (str): End date in YYYY-MM-DD format.
        interval (int, optional): Interval between dates in days. Defaults to 1.
    
    Returns:
        Iterator[date]: Iterator of dates between start_date and end_date.
    """

    start_date = date.fromisoformat(start_date)
    end_date = date.fromisoformat(end_date)
    delta = end_date - start_date
    for i in range(0, delta.days + 1, interval):
        yield start_date + timedelta(days=i)

In [5]:
# get all stocks in the S&P 500 since 2013
sp500 = set()
for date in daterange("2013-01-01", "2024-04-08"):
    sp500.update(lookups.get_sp500_tickers(str(date)))

In [6]:
len(sp500)

750

In [12]:
failed = set()
for ticker in sp500:
    try:
        processor.process_sec_json(ticker)
    except Exception as e:
        failed.add(ticker)

In [13]:
len(failed)

151

### 1. Explore stocks for which we don't have processed financial data

Most likely has to do with an error during translation of the ticker symbol to the company's CIK number, since the downloaded data from the SEC is stored by CIK number. Look for historical ticker to CIK mappings. Feel free to print error messages to the notebook to see what went wrong.

In [15]:
failed = set()
for ticker in sp500:
    try:
        s = stock.Stock(ticker)
    except:
        failed.add(ticker)

In [16]:
len(failed)

150

### 2. Explore stocks for which we don't have UFCF data

UFCF = ebit - tax_expense + d_and_a - capex - change_in_wc

If any one of the above terms cannot be found, UFCF will be NaN for that stock. So, we need to explore the stocks for which we don't have UFCF data and see if we can find the missing terms.

Once missing terms are located, use Google / Yahoo Finance / SEC filings to see what the correct value should be (e.g. search for "AAPL capital expenditures 2019" on Yahoo Finance). Now that we know what the correct value should be, we need to find what XRBL tag that value is under.

To find the XRBL tag, open the .csv file for the stock and search for the value. The XRBL tag will be the column name. Ensure that the date and XRBL tag name make sense. If they do, add the tag to the list of tags in the "get_{term}" function in stock.py.

Start with DATE=None to get latest data. Once the coverage of UFCF has improved to >= 90% coverage of stocks, change to an earlier date (e.g. DATE="2022-03-28") and explore the stocks for which we don't have UFCF data. Continue this process of improving coverage and moving back the date as far as you can -- hopefully until 2013.

In [35]:
DATE = None

In [38]:
# may want to add additional columns for the components of certain metrics
# e.g. if change in working capital is missing for a lot of stocks, consider adding
# a column for current assets and current liabilities for both the current and previous year
data = {}
for ticker in tqdm(sp500 - failed):
    s = stock.Stock(ticker)
    try:
        ufcf = s.get_ufcf(DATE)
    except:
        ufcf = None
    try:
        ebit = s.get_ebit(DATE)
    except:
        ebit = None
    try:
        tax_expense = s.get_tax_expense(DATE)
    except:
        tax_expense = None
    try:
        depreciation = s.get_depreciation(DATE)
    except:
        depreciation = None
    try:
        amortization = s.get_amortization(DATE)
    except:
        amortization = None
    try:
        d_and_a = s.get_depreciation_and_amortization(DATE)
    except:
        d_and_a = None
    try:
        capex = s.get_capex(DATE)
    except:
        capex = None
    try:
        change_in_wc = s.get_change_in_working_capital(DATE)
    except:
        change_in_wc = None
    data[ticker] = {
        "ufcf": ufcf,
        "ebit": ebit,
        "tax_expense": tax_expense,
        "depreciation": depreciation,
        "amortization": amortization,
        "d_and_a": d_and_a,
        "capex": capex,
        "change_in_wc": change_in_wc,
    }

100%|██████████| 568/568 [00:55<00:00, 10.33it/s]


In [39]:
df = pd.DataFrame(data).T

In [40]:
df[df["ufcf"].isna()]

,ufcf,ebit,tax_expense,depreciation,amortization,d_and_a,capex,change_in_wc
UDR,NaN,250814000.0,349000.0,NaN,NaN,14344000.0,NaN,-80567000.0
PX,None,None,None,None,None,None,None,None
BHF,NaN,-24000000.0,-182000000.0,NaN,NaN,NaN,NaN,-297000000.0
APA,NaN,5565000000.0,1652000000.0,NaN,2000000.0,1233000000.0,NaN,-471000000.0
EQR,NaN,1116046000.0,900000.0,882168000.0,5004000.0,887172000.0,NaN,-423949000.0
...,...,...,...,...,...,...,...,...
FITB,NaN,4071000000.0,647000000.0,121000000.0,48000000.0,436000000.0,348000000.0,NaN
SYF,NaN,5483000000.0,946000000.0,NaN,NaN,419000000.0,NaN,4858000000.0
UAA,None,None,None,None,None,None,None,None
NTRS,NaN,2756800000.0,430300000.0,NaN,9300000.0,553600000.0,128600000.0,NaN


In [41]:
df[df["capex"].isna()]

,ufcf,ebit,tax_expense,depreciation,amortization,d_and_a,capex,change_in_wc
UDR,NaN,250814000.0,349000.0,NaN,NaN,14344000.0,NaN,-80567000.0
PX,None,None,None,None,None,None,None,None
BHF,NaN,-24000000.0,-182000000.0,NaN,NaN,NaN,NaN,-297000000.0
APA,NaN,5565000000.0,1652000000.0,NaN,2000000.0,1233000000.0,NaN,-471000000.0
EQR,NaN,1116046000.0,900000.0,882168000.0,5004000.0,887172000.0,NaN,-423949000.0
WRB,NaN,1846163000.0,334727000.0,52000000.0,NaN,55872000.0,NaN,-132779000.0
LOW,None,None,None,None,None,None,None,None
FANG,NaN,6508000000.0,1174000000.0,NaN,NaN,1344000000.0,NaN,-332000000.0
GNW,NaN,633000000.0,239000000.0,NaN,19000000.0,307000000.0,NaN,226000000.0
SNA,NaN,1207200000.0,268700000.0,71500000.0,28700000.0,100200000.0,NaN,326100000.0


### 3. Explore stocks for which we don't have debt, cash, and shares outstanding data

Follow the instructions in the section above, but for debt, cash, and shares outstanding.

Start with DATE=None to get latest data. Once the coverage of UFCF has improved to >= 90% coverage of stocks, change to an earlier date (e.g. DATE="2022-03-28") and explore the stocks for which we don't have UFCF data. Continue this process of improving coverage and moving back the date as far as you can -- hopefully until 2013.

In [42]:
DATE = None

In [43]:
data = {}
for ticker in tqdm(sp500 - failed):
    s = stock.Stock(ticker)
    try:
        total_debt = s.get_total_debt(DATE)
    except:
        total_debt = None
    try:
        current_debt = s.get_current_debt(DATE)
    except:
        current_debt = None
    try:
        noncurrent_debt = s.get_noncurrent_debt(DATE)
    except:
        noncurrent_debt = None
    try:
        cash = s.get_cash(DATE)
    except:
        cash = None
    try:
        shares_outstanding = s.get_shares_outstanding(DATE)
    except:
        shares_outstanding = None
    data[ticker] = {
        "total_debt": total_debt,
        "current_debt": current_debt,
        "noncurrent_debt": noncurrent_debt,
        "cash": cash,
        "shares_outstanding": shares_outstanding,
    }

100%|██████████| 568/568 [00:30<00:00, 18.49it/s]


In [44]:
df = pd.DataFrame(data).T

Notice below that not a lot of tickers are missing data. This is because we are using the latest data, which is more likely to be complete. As we move back in time, the coverage of data will decrease. Try to get as much coverage as possible.

In [45]:
df[df["total_debt"].isna()]

,total_debt,current_debt,noncurrent_debt,cash,shares_outstanding
PX,None,None,None,None,None
RJF,NaN,NaN,NaN,6178000000.0,215063590.0
LEN,NaN,NaN,NaN,4815770000.0,289824000.0
AIZ,NaN,NaN,NaN,1536700000.0,52919741.0
WRB,NaN,NaN,NaN,1449346000.0,263446321.0
PCAR,NaN,NaN,NaN,4690900000.0,522513846.0
LOW,None,None,None,None,None
MPWR,NaN,NaN,NaN,273145000.0,47305000.0
TROW,NaN,NaN,NaN,1755600000.0,224398924.0
AMP,NaN,0.0,NaN,8755000000.0,105279357.0


In [46]:
df[df["cash"].isna()]

,total_debt,current_debt,noncurrent_debt,cash,shares_outstanding
PX,None,None,None,None,None
LOW,None,None,None,None,None
FL,None,None,None,None,None
HCP,None,None,None,None,None
GME,None,None,None,None,None
UAA,None,None,None,None,None


In [47]:
df[df["shares_outstanding"].isna()]

,total_debt,current_debt,noncurrent_debt,cash,shares_outstanding
PX,None,None,None,None,None
LOW,None,None,None,None,None
FOXA,7206000000.0,0.0,7206000000.0,5200000000.0,NaN
TSN,8321000000.0,459000000.0,350000000.0,1031000000.0,NaN
FL,None,None,None,None,None
FOX,7206000000.0,0.0,7206000000.0,5200000000.0,NaN
HCP,None,None,None,None,None
V,22450000000.0,2250000000.0,20200000000.0,15689000000.0,NaN
ADT,9937368000.0,871917000.0,8956671000.0,257223000.0,NaN
STZ,10093500000.0,605300000.0,9488200000.0,199400000.0,NaN
